### Weather API
Documentacion : 

se puede bajar un excel 
la latitud y longitud de nueva yor es un requisito de la api estas son_ 
long: -74.0059700 lat: 40.7142700

EN EL SIGUIENTE CODIGO CON AYUDA DE LA PAGINA QUE NOS PROPORCIONA LA API OPEN-METEO. DOS DATAFRAMES, POR HORA Y DIA.

EN EL DE HORAS, TRAEMOS POR HORA TEMPERATURA, LLUVIA, CAIDA DE NIEVE, PROFUNDIDAD DE NIEVE, SI ES DIA O NO
"hourly": ["temperature_2m", "rain", "snowfall", "snow_depth", "is_day"],

EN EL DE DIA TRAEMOS, LA MEDIA DE TEMPERATURA Y LA CANTIDAD DE LLUVIA.
"daily": ["temperature_2m_mean", "rain_sum"],

EN UN LAPSO DE TIMEPO DESDE 01/01/2010 HASTA 31/12/2019



In [1]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 40.71,
	"longitude": -74,
	"start_date": "2010-01-01",
	"end_date": "2022-12-31",
	"hourly": ["temperature_2m", "rain", "snowfall", "snow_depth", "is_day"],
	"daily": ["temperature_2m_mean", "rain_sum"],
	"timezone": "America/Sao_Paulo"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_rain = hourly.Variables(1).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(2).ValuesAsNumpy()
hourly_snow_depth = hourly.Variables(3).ValuesAsNumpy()
hourly_is_day = hourly.Variables(4).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s"),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s"),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["rain"] = hourly_rain
hourly_data["snowfall"] = hourly_snowfall
hourly_data["snow_depth"] = hourly_snow_depth
hourly_data["is_day"] = hourly_is_day

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_rain_sum = daily.Variables(1).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s"),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s"),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["rain_sum"] = daily_rain_sum

daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)

Coordinates 40.738136291503906°E -74.04254150390625°N
Elevation 7.0 m asl
Timezone b'America/Sao_Paulo' b'-03'
Timezone difference to GMT+0 -10800 s
                      date  temperature_2m  rain  snowfall  snow_depth  is_day
0      2010-01-01 03:00:00         -1.0805   0.1      0.21        0.03     0.0
1      2010-01-01 04:00:00         -1.5305   0.0      0.35        0.03     0.0
2      2010-01-01 05:00:00         -1.4305   0.1      0.28        0.04     0.0
3      2010-01-01 06:00:00         -0.9805   0.1      0.21        0.04     0.0
4      2010-01-01 07:00:00         -0.7305   0.1      0.14        0.04     0.0
...                    ...             ...   ...       ...         ...     ...
113947 2022-12-31 22:00:00          9.7585   0.2      0.00        0.00     0.0
113948 2022-12-31 23:00:00         10.5585   0.2      0.00        0.00     0.0
113949 2023-01-01 00:00:00          8.9585   0.4      0.00        0.00     0.0
113950 2023-01-01 01:00:00         11.1085   1.1      0.00   

In [3]:
daily_dataframe.shape

(3652, 3)

In [4]:
daily_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3652 entries, 0 to 3651
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 3652 non-null   datetime64[ns]
 1   temperature_2m_mean  3652 non-null   float32       
 2   rain_sum             3652 non-null   float32       
dtypes: datetime64[ns](1), float32(2)
memory usage: 57.2 KB


In [5]:
daily_dataframe.head()

,date,temperature_2m_mean,rain_sum
0,2010-01-01 03:00:00,0.752833,0.4
1,2010-01-02 03:00:00,-2.924250,0.1
2,2010-01-03 03:00:00,-7.561750,0.0
3,2010-01-04 03:00:00,-4.707583,0.0
4,2010-01-05 03:00:00,-4.486750,0.0


In [6]:
hourly_dataframe.shape

(87648, 6)

In [7]:
hourly_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87648 entries, 0 to 87647
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            87648 non-null  datetime64[ns]
 1   temperature_2m  87648 non-null  float32       
 2   rain            87648 non-null  float32       
 3   snowfall        87648 non-null  float32       
 4   snow_depth      87648 non-null  float32       
 5   is_day          87648 non-null  float32       
dtypes: datetime64[ns](1), float32(5)
memory usage: 2.3 MB


In [8]:
hourly_dataframe.head()

,date,temperature_2m,rain,snowfall,snow_depth,is_day
0,2010-01-01 03:00:00,-1.0805,0.1,0.21,0.03,0.0
1,2010-01-01 04:00:00,-1.5305,0.0,0.35,0.03,0.0
2,2010-01-01 05:00:00,-1.4305,0.1,0.28,0.04,0.0
3,2010-01-01 06:00:00,-0.9805,0.1,0.21,0.04,0.0
4,2010-01-01 07:00:00,-0.7305,0.1,0.14,0.04,0.0


In [18]:
hourly_dataframe.describe()

,date,temperature_2m,rain,snowfall,snow_depth,is_day
count,87648,87648.000000,87648.000000,87648.000000,87648.000000,87648.000000
mean,2015-01-01 02:30:00,12.226660,0.119443,0.009074,0.017160,0.507827
min,2010-01-01 03:00:00,-20.441500,0.000000,0.000000,0.000000,0.000000
25%,2012-07-02 02:45:00,4.019500,0.000000,0.000000,0.000000,0.000000
50%,2015-01-01 02:30:00,12.708500,0.000000,0.000000,0.000000,1.000000
75%,2017-07-02 02:15:00,20.808500,0.000000,0.000000,0.000000,1.000000
max,2020-01-01 02:00:00,38.769501,15.900000,4.340000,0.470000,1.000000
std,NaN,10.283067,0.528260,0.099267,0.058375,0.499942


In [20]:
hourly_dataframe.is_day.unique()

array([0., 1.], dtype=float32)

In [21]:
hourly_dataframe[hourly_dataframe['is_day']==1]

,date,temperature_2m,rain,snowfall,snow_depth,is_day
10,2010-01-01 13:00:00,-0.4805,0.0,0.0,0.04,1.0
11,2010-01-01 14:00:00,0.3695,0.0,0.0,0.04,1.0
12,2010-01-01 15:00:00,1.2695,0.0,0.0,0.04,1.0
13,2010-01-01 16:00:00,2.6695,0.0,0.0,0.04,1.0
14,2010-01-01 17:00:00,2.2195,0.0,0.0,0.04,1.0
...,...,...,...,...,...,...
87638,2019-12-31 17:00:00,4.7585,0.0,0.0,0.00,1.0
87639,2019-12-31 18:00:00,5.2585,0.0,0.0,0.00,1.0
87640,2019-12-31 19:00:00,5.9585,0.0,0.0,0.00,1.0
87641,2019-12-31 20:00:00,6.7585,0.0,0.0,0.00,1.0


In [22]:
hourly_dataframe.to_excel('tiempo_por_hora.xlsx')